# Prepare SWIFT data
This notebook combines SWIFT buoy data with data from other sources. SWIFT buoy data has been preprocessed using [SWIFT-codes](https://github.com/SASlabgroup/SWIFT-codes) and converted to python-friendly format using `read_swift` in [swifttools.py](https://github.com/SASlabgroup/SWIFT-codes/blob/master/Python/swifttools.py) beforehand.

This notebook will add the following data:

1. co-located sea ice concentration
2. energy weighted mean direction of the swell band
3. distance to the ice edge (shortest and in swell wave direction)
4. ice codes from manual classification of SWIFT onboard imagery

In [ ]:
import pandas as pd
import xarray as xr
import os

import colocation as co
import icegeometry as ig
import waves

In [ ]:
deployment = 'winter'
swift_data_folder = # fill in folder
save_folder = '../processed_data/'

if deployment == 'winter':
    swift20  = pd.read_feather(os.path.join(swift_data_folder, 'swift20_winter.feather'))
    swift21  = pd.read_feather(os.path.join(swift_data_folder, 'swift21_winter.feather'))  
    sic      = xr.open_dataset('../data/sic_winter.nc')    
    ice_edge = pd.read_feather('../processed_data/ice_edge_winter_sigma1.feather')
    region   = pd.read_csv('../processed_data/ice_edge_winter_sigma1_region.csv')    
    icetype_swift20 = None
    icetype_swift21 = '../data/icecode_swift21_winter.csv'
    
elif deployment == 'spring':
    swift20  = pd.read_feather(os.path.join(swift_data_folder, 'swift20_spring.feather'))
    swift21  = pd.read_feather(os.path.join(swift_data_folder, 'swift21_spring.feather'))   
    sic      = xr.open_dataset('../data/sic_spring.nc')    
    ice_edge = pd.read_feather('../processed_data/ice_edge_spring_sigma1.feather')
    region   = pd.read_csv('../processed_data/ice_edge_spring_sigma1_region.csv')    
    icetype_swift20 = '../data/icecode_swift20_spring.csv'
    icetype_swift21 = '../data/icecode_swift21_spring.csv'
    
swift_deployments = [swift20, swift21]
icetype_files = [icetype_swift20, icetype_swift21]
labels = ['swift20', 'swift21']

## Co-locate with sea ice concentration

In [ ]:
for swift in swift_deployments:
    # Convert lon-lat to polar stereographic xy-coordinates 
    # (since sea ice data is in polare stereographic coordinates)
    # and add to swift df
    co.add_xy_to_swift(swift, sic)

    # Colocate ice conecentration with swift location
    co.swift_collocate_sic(swift, sic, method = 'nearest', column = 'sic')

## Add wave band properties

In [ ]:
f_start  = 0.05 # Hz
f_stop   = 0.13 # Hz
bandname = 'swell'

for swift in swift_deployments:
    waves.swift_add_waveband_properties(swift, f_start = f_start, f_stop = f_stop, bandname = 'swell')

## Get distance to the sea ice edge

In [ ]:
for swift in swift_deployments:
    # Closest distance to edge
    ig.add_distance_to_edge(swift, ice_edge, region, "closest", "closest")

    # Distance to edge in wave direction
    ig.add_distance_to_edge(swift, ice_edge, region, "swell.meandirT", "wavedir")

## Add ice codes

In [ ]:
for (swift, icetype) in zip(swift_deployments, icetype_files):
    co.swift_add_icetype(swift, icetype)

## Save SWIFT

In [ ]:
for (swift, label) in zip(swift_deployments, labels):
    path = os.path.join(save_folder, label + '_' + deployment + '.feather')
    swift.to_feather(path)